# MNIST Image Classification with Keras and TensorFlow: A Tutorial

## 1. Introduction
Welcome! This notebook provides a guide to a Python script that trains a neural network to recognize handwritten digits from the famous MNIST dataset. This is a classic machine learning project, excellent for beginners.
The script (`main.py`) performs several key operations:
1. Loads the MNIST dataset (70,000 images of handwritten digits 0-9).
2. Builds a Multi-Layer Perceptron (MLP) neural network using Keras with TensorFlow.
3. Trains the model to classify the digit images.
4. Visualizes the training process and model architecture using TensorBoard and Visualkeras.
5. Evaluates the model's performance and makes predictions on new images.

## 📺 Watch the Tutorial

Prefer a video walkthrough? Check out the accompanying tutorial on YouTube:

[Image classification with MLP](https://youtu.be/dkZ3sS_zqog)

## 2. Core Concepts for Beginners

### MNIST Dataset
The MNIST dataset is a collection of 70,000 grayscale images, each 28x28 pixels, showing a single handwritten digit (0-9). It contains 60,000 images for training and 10,000 for testing. It's a benchmark dataset for image classification.

### Image Classification
This is the primary task. Given an input image (like a handwritten digit), the goal is to assign it a category or label (in this case, the digit it represents, 0-9).

### Multi-Layer Perceptron (MLP) vs. Convolutional Neural Networks (CNNs)
While **CNNs** are specialized for image data, recognizing patterns like edges and shapes by processing data in its grid-like topology, this script uses a simpler **MLP**.
**Why an MLP here?** The MNIST dataset's simplicity (small, centered digits) allows an MLP to perform reasonably well and serve as a good introduction to neural networks. 
**Key Difference:** An MLP flattens the 2D image (28x28 pixels) into a 1D vector (784 numbers). It then processes this vector through fully connected layers. This flattening means the MLP doesn't inherently preserve spatial relationships between pixels. CNNs, using convolutional layers, explicitly preserve these spatial structures, making them more powerful for complex image tasks. For understanding basics with MNIST, an MLP is a suitable starting point.

### Key Steps in the Machine Learning Project
1.  **Data Loading and Preprocessing:** Getting data ready for the model.
2.  **Model Building:** Defining the neural network's architecture.
3.  **Model Compilation:** Configuring the learning process (optimizer, loss function, metrics).
4.  **Training:** Feeding data to the model so it can learn.
5.  **Evaluation:** Checking model performance on unseen data.
6.  **Prediction:** Using the trained model on new data.

### Visualization Tools: TensorBoard and Visualkeras
*   **TensorBoard:** TensorFlow's visualization toolkit for monitoring training metrics (accuracy, loss), visualizing the model graph, and examining weights.
*   **Visualkeras:** A Python library for creating clear, layered diagrams of Keras model architectures.

## 3. Code Deep Dive: `main.py`

### File Structure Overview
The entire logic is within `main.py`, defining several functions for different parts of the process and a `main()` function to orchestrate them.

### The Imports Section
These lines bring in all necessary tools and libraries.
```python
import tensorflow as tf
from tensorflow.keras.layers import (
    Input,
    Dense,
    Dropout,
    BatchNormalization,
    Activation,
    Flatten,
)
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
import visualkeras
```

### Function: `load_and_preprocess_data()`
Responsible for getting and preparing the MNIST dataset.
```python
def load_and_preprocess_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0
    y_train = to_categorical(y_train, num_classes=10)
    y_test = to_categorical(y_test, num_classes=10)
    return (x_train, y_train), (x_test, y_test)
```
*   `mnist.load_data()`: Downloads and loads the MNIST dataset (training and testing images/labels).
*   `astype("float32") / 255.0`: Converts pixel values to `float32` and normalizes them to the range [0, 1]. Normalization helps training converge faster and more reliably.
*   `to_categorical(...)`: Performs one-hot encoding on labels (e.g., `5` becomes `[0,0,0,0,0,1,0,0,0,0]`). This is needed for `categorical_crossentropy` loss.

### Function: `build_model(input_shape=(28, 28), num_classes=10)`
Defines the MLP architecture using Keras Functional API.
```python
def build_model(input_shape=(28, 28), num_classes=10):
    inputs = Input(shape=input_shape, name="mnist_input")
    x = Flatten(name="flatten")(inputs)
    x = Dense(128, name="dense_1")(x)
    x = Activation("relu", name="act_1")(x)
    x = BatchNormalization(name="bn_1")(x)
    x = Dropout(0.5, name="dropout_1")(x)
    x = Dense(64, name="dense_2")(x)
    x = Activation("relu", name="act_2")(x)
    x = BatchNormalization(name="bn_2")(x)
    x = Dropout(0.5, name="dropout_2")(x)
    outputs = Dense(num_classes, activation="softmax", name="predictions")(x)
    return Model(inputs=inputs, outputs=outputs, name="mnist_mlp")
```
*   **Keras Functional API:** Allows building complex layer graphs.
*   `Input(...)`: Defines the input layer (28x28 images for MNIST).
*   `Flatten(...)`: Converts 2D image data to 1D for Dense layers.
*   **Dense Blocks:** Consist of:
    *   `Dense(...)`: Fully connected layer (128 neurons, then 64 neurons).
    *   `Activation("relu", ...)`: ReLU activation function (`f(x) = max(0,x)`), helps with vanishing gradients.
    *   `BatchNormalization(...)`: Normalizes previous layer's output, stabilizing learning and speeding up training.
    *   `Dropout(0.5, ...)`: Regularization technique to prevent overfitting by randomly dropping 50% of neurons during training.
*   **Output Layer:** `Dense(num_classes, activation="softmax", ...)`: Final layer with 10 neurons (one for each digit) and `softmax` activation to output a probability distribution across classes.
*   `Model(...)`: Creates the Keras model object.

### Function: `predict_and_save_samples(...)`
Makes predictions on test samples and saves them as images.
```python
def predict_and_save_samples(model, x_test, y_test, num_samples=5, output_dir="predictions"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    sample_indices = np.random.choice(x_test.shape[0], num_samples, replace=False)
    images = x_test[sample_indices]
    labels = y_test[sample_indices]
    preds = model.predict(images)
    pred_classes = np.argmax(preds, axis=1)
    true_classes = np.argmax(labels, axis=1)
    print(f"\n--- Predictions on {num_samples} samples ---")
    for i, img in enumerate(images):
        print(f"Sample {i+1}: Predicted={pred_classes[i]}, True={true_classes[i]}")
        img_plot = img.reshape(28,28) # Reshape flattened image back to 2D for saving
        path = os.path.join(output_dir, f"sample_{i+1}_true_{true_classes[i]}_pred_{pred_classes[i]}.png")
        plt.imsave(path, img_plot, cmap='gray')
        print(f"  Saved image to: {path}")
    print("-----------------------------------------")
```
*   Creates an `output_dir` if it doesn't exist.
*   Randomly selects `num_samples` from `x_test`.
*   `model.predict(images)`: Gets predictions (softmax probabilities) from the model.
*   `np.argmax(...)`: Converts probabilities and one-hot labels to class indices (predicted and true digits).
*   Loops through samples, prints predictions, reshapes images to 28x28, and saves them using `plt.imsave`.

### Function: `main()`
Orchestrates the entire machine learning workflow.
```python
def main():
    # Load data
    (x_train, y_train), (x_test, y_test) = load_and_preprocess_data()

    # Build and compile model
    model = build_model()
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    model.summary() # Prints model summary

    # Prepare TensorBoard log directory (timestamped)
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    logdir = os.path.join("logs", "graphs", timestamp)
    os.makedirs(logdir, exist_ok=True)

    # Generate and save model architecture using Visualkeras
    arch_path = os.path.join(logdir, 'model_visualkeras.png')
    visualkeras.layered_view(
        model,
        to_file=arch_path,
        legend=True, draw_volume=False, scale_xy=1.5, scale_z=1, spacing=20
    )

    # Log Visualkeras image to TensorBoard
    with tf.summary.create_file_writer(logdir).as_default():
        img_data = tf.io.read_file(arch_path)
        img_tensor = tf.image.decode_png(img_data, channels=4)
        tf.summary.image("Model Visualization", tf.expand_dims(img_tensor, 0), step=0)

    # TensorBoard callback setup
    tensorboard_cb = tf.keras.callbacks.TensorBoard(
        log_dir=logdir,
        histogram_freq=1,       # Log histograms every epoch
        write_graph=True,       # Log model graph
        write_images=True,      # Log model weights as images (if applicable)
        update_freq='epoch',    # Update scalars every epoch
        profile_batch=1         # Enable profiler for batch 1
    )
    print(f"TensorBoard logs in: {os.path.abspath(logdir)}")
    print("Run: tensorboard --logdir logs")

    # Train model
    model.fit(
        x_train, y_train,
        batch_size=128,         # Number of samples per gradient update
        epochs=10,              # Number of times to iterate over the entire training dataset
        validation_split=0.1,   # Fraction of training data for validation
        callbacks=[tensorboard_cb], # Callback for TensorBoard logging
        verbose=2               # Print one line per epoch
    )

    # Evaluate model on test data
    loss, acc = model.evaluate(x_test, y_test, verbose=0)
    print(f"\nTest accuracy: {acc:.4f}, loss: {loss:.4f}")

    # Predict and save sample outputs
    predict_and_save_samples(model, x_test, y_test)
```
**Breakdown:**
*   **Load Data:** Calls `load_and_preprocess_data()`.
*   **Build and Compile Model:** Calls `build_model()`. Then, `model.compile(...)` configures training with:
    *   `optimizer="adam"`: Adam optimizer.
    *   `loss="categorical_crossentropy"`: Loss function for one-hot encoded multi-class classification.
    *   `metrics=["accuracy"]`: Metric to monitor.
    *   `model.summary()`: Prints model layers and parameters.
*   **Prepare TensorBoard Log Directory:** Creates a timestamped directory for logs.
*   **Generate Visualkeras Architecture:** Uses `visualkeras.layered_view()` to create and save a diagram of the model.
*   **Log Visualkeras Image to TensorBoard:** Reads the saved diagram and logs it to TensorBoard using `tf.summary.image`.
*   **TensorBoard Callback:** `tf.keras.callbacks.TensorBoard(...)` logs metrics, graphs, histograms, etc., during training.
*   **Train Model:** `model.fit(...)` starts training with specified batch size, epochs, validation split, and callbacks.
*   **Evaluate:** `model.evaluate(...)` calculates loss and accuracy on the unseen test set.
*   **Predict and Save:** Calls `predict_and_save_samples()`.

### `if __name__ == "__main__":`
Ensures `main()` runs only when the script is executed directly.
```python
if __name__ == "__main__":
    main()
```

## 4. Setup and Running the Application

### Prerequisites
*   Python 3 (e.g., 3.8+)
*   `pip` (Python package installer)

### Installation Steps
1.  **Clone the repository (if applicable):**
    ```bash
    # git clone <your-repo-url>
    # cd Keras/FuncApiExample  (Adjust path as needed for your project structure)
    ```
2.  **Create a Python virtual environment:** (Highly recommended)
    ```bash
    python3 -m venv venv
    ```
3.  **Activate the virtual environment:**
    *   Linux/macOS:
        ```bash
        source venv/bin/activate
        ```
    *   Windows (PowerShell):
        ```bash
        .\venv\Scripts\Activate.ps1
        ```
4.  **Install the required packages:** Ensure you have a `requirements.txt` file similar to this (based on the script's imports):
    ```txt
    tensorflow
    numpy
    matplotlib
    visualkeras
    ```
    Then run:
    ```bash
    pip install -r requirements.txt
    ```
5.  **Install Graphviz (for Visualkeras to save complex model graphs, especially if not just using `to_file` with simple formats):**
    Visualkeras might require Graphviz for some functionalities or for saving in formats like PDF or SVG directly. For PNG as used in the script, it might not be strictly necessary if its internal rendering is sufficient, but it's good practice to have it installed for full Keras/TensorFlow visualization capabilities.
    *   **Debian/Ubuntu Linux:**
        ```bash
        sudo apt-get update
        sudo apt-get install graphviz
        ```
    *   For other systems, refer to Graphviz installation guides.

### Running the Project
Once setup is complete, run the main script (assuming it's named `main.py`):
```bash
python main.py
```
This will:
- Load and preprocess the MNIST dataset.
- Build the Keras Functional API model.
- Train the model.
- Evaluate the model on the test set.
- Predict on a few random samples and save the images with their true and predicted labels in a `predictions` directory.

### Starting TensorBoard
To view the logs (metrics, graph, model visualization):
```bash
tensorboard --logdir logs
```
Then open the provided URL (usually `http://localhost:6006`) in your web browser.

### Deactivating the Virtual Environment
When finished:
```bash
deactivate
```

## 5. Conclusion
This notebook has walked through a Python script for MNIST image classification using Keras and TensorFlow. We've covered data loading, preprocessing, model building (MLP), compilation, training, visualization with TensorBoard and Visualkeras, evaluation, and prediction.
This script provides a solid foundation for understanding image classification with MLPs. From here, you can experiment with different network architectures, layers, neurons, optimizers, or try it on different datasets!